# Explore peak & hwhm plotting 

In [ ]:
import pandas as pd 
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches
import re

def get_textbox_yminmax(figure_handle,axes_handle,textbox_handle):
    transf = axes_handle.transAxes.inverted()
    bb = textbox_handle.get_window_extent(renderer = figure_handle.canvas.get_renderer())
    bb_datacoords = bb.transformed(transf)
    return [bb_datacoords.ymin,bb_datacoords.ymax]


In [ ]:
parent_folder = '/Users/alexvourvoukelis/Desktop/DTP_rotation_2/Code/L5/simulations'

simulation_dirs = []
peak_hwhm_files = []
for root, directories, files in os.walk(parent_folder, topdown=False):
    for name in files:
        if 'population_peaks' in name:
            peak_hwhm_files.append(os.path.join(root, name))
    for name in directories:
        if 'temporaloffset' in name:
            simulation_dirs.append(os.path.join(root, name))

In [ ]:
criteria = ['L5a-100_L5bvaried','discrete']
peaks_csv_path = [s for s in peak_hwhm_files if all(x in s for x in criteria)][0]

#### Title builder

In [ ]:
simulation_label = peaks_csv_path.split('/')[-2]

simulation_label_split = simulation_label.split('_')
model_title = simulation_label_split[0].capitalize()
if 'temporaloffset' in simulation_label_split:
    if 'asynchronous' in simulation_label_split:
        synchronicity_title = 'asynchronous'
        if 'L5bvaried' in simulation_label_split:
            L5a_label = [i for i in simulation_label_split if 'L5a' in i][-1]
            L5a_title = 'L5a locked ('+L5a_label[3:]+'ms)'
            L5b_title = 'L5b varied'
        if 'L5avaried' in simulation_label_split:
            L5a_title = 'L5a varied'
            L5b_label = [i for i in simulation_label_split if 'L5b' in i][-1]
            L5b_title = 'L5b locked ('+L5b_label[3:]+'ms)'
    if 'synchronous' in simulation_label_split:
        synchronicity_title = 'synchronous'
        
general_simulation_title = model_title+' model ('+synchronicity_title+' temporal offset)'
general_simulation_subtitle = L5a_title+' - '+L5b_title
general_simulation_subtitle = general_simulation_subtitle.rjust(int(0.75*len(general_simulation_title)), ' ')
general_simulation_title = general_simulation_title+'\n'+general_simulation_subtitle        


In [ ]:
general_simulation_title

In [ ]:
# peaks_csv_path = '/Users/alexvourvoukelis/Desktop/DTP_rotation_2/Code/L5/simulations/homogeneous_synchronous_temporaloffset_10ms_200mswindow_250msdutycycle/population_peaks.csv'
# general_simulation_title = 'Homogeneous synchronous'
# simulation_label = 'homogeneous_synchronous_temporaloffset_10ms_200mswindow_250msdutycycle'
# parent_folder = '/Users/alexvourvoukelis/Desktop/DTP_rotation_2/Code/L5/simulations'


In [ ]:
test_df = pd.read_csv(peaks_csv_path)
epochs = test_df['epoch'].drop_duplicates()
epochs = epochs.to_list()

In [ ]:
pd.set_option('display.max_rows', None)
test_df

In [ ]:


populations = ['L2/3']
populations_multiindex = np.repeat(populations,len(epochs))

epochs_multiindex = np.tile(epochs,1)

multiindex_plot = pd.MultiIndex.from_arrays([populations_multiindex,epochs_multiindex],names=('population', 'epoch') )

for i,measure in enumerate(['amplitude','hwhm']):

    mean_s = test_df.set_index('epoch').groupby(['population', 'epoch'])[measure].mean()
    mean_s = mean_s.reindex(multiindex_plot)
    sem_s = test_df.set_index('epoch').groupby(['population', 'epoch'])[measure].sem()
    sem_s = sem_s.reindex(multiindex_plot)

    mean_df = mean_s.to_frame()
    mean_df['sem'] = sem_s
    mean_df_plot = mean_df.reset_index()

    if measure == 'amplitude': 
        mean_amplitude_df = mean_df_plot
    if measure == 'hwhm': 
        mean_hwhm_df = mean_df_plot
    
mean_hwhm_df = mean_hwhm_df.loc[mean_hwhm_df['population']=='L2/3']
mean_amplitude_df = mean_amplitude_df.loc[mean_amplitude_df['population']=='L2/3']



In [ ]:

# fig, ax = plt.subplots(figsize = (18,5), sharey=True)

# x_axis = np.arange(0,len(epochs))
# ax2 = ax.twinx()
# ax.errorbar(x_axis, mean_amplitude_df['amplitude'],fmt='D',yerr= mean_amplitude_df['sem'], color = 'g')
# ax2.plot(x_axis, mean_hwhm_df['hwhm'], 'b-')
# # plt.errorbar(x, y, yerr=e, fmt='o')


# ax.set_xlabel('X data')
# ax.set_ylabel('Y1 data', color='g')
# ax2.set_ylabel('Y2 data', color='b')


# ax.set_xticks(x_axis) # REMOVE IN THE FUTURE - PLACED TO AVOID WARNING - IT IS A BUG FROM MATPLOTLIB 3.3.1
# ax.set_xticklabels(epochs)
# ax.tick_params(axis='x', labelrotation= 25)
 
# # # add legend --> take advantage of pandas providing us access
# # # to the line associated with the right part of the axis
# # ax.legend([ax.get_lines()[0], ax.right_ax.get_lines()[0], ax3.get_lines()[0]],\
# #            ['A','B','C'], bbox_to_anchor=(1.5, 0.5))

In [ ]:

fig = plt.figure(figsize = (18,7))
ax = plt.axes()
# ax2 = ax1.twinx()
# rspine = ax2.spines['right']
# rspine.set_position(('axes', 1.15))
# ax2.set_frame_on(True)
# ax2.patch.set_visible(False)


# same ax as above since it's automatically added on the right
mean_hwhm_df.hwhm.plot(ax=ax, color='tab:orange', secondary_y=True,yerr=mean_hwhm_df['sem'])
mean_amplitude_df.amplitude.plot(ax=ax, color='tab:blue',yerr=mean_amplitude_df['sem'])
# ax.legend([ax.get_lines()[0], ax.right_ax.get_lines()[0]],\
#            ['A','B'], bbox_to_anchor=(1.25, 0.65))
ax.set_ylabel('Amplitude (pA)',fontsize=15,labelpad=15)
ax.yaxis.label.set_color('tab:blue')
ax.right_ax.set_ylabel('Half-width half-maximum (ms)',fontsize=15,labelpad=15)
ax.right_ax.yaxis.label.set_color('tab:orange')

ax.set_xticks(np.arange(0,len(epochs))) # REMOVE IN THE FUTURE - PLACED TO AVOID WARNING - IT IS A BUG FROM MATPLOTLIB 3.3.1
ax.set_xticklabels(epochs,fontsize=12)
plt.setp( ax.xaxis.get_majorticklabels(), rotation=45, ha="right" )

fig.suptitle(general_simulation_title,fontsize=25)
plt.tight_layout()
plt.show()


fig.savefig("/".join((parent_folder,simulation_label,'amplitude_hwhm_lineplot.pdf')))


In [ ]:

fig = plt.figure(figsize = (18,7))
ax = plt.axes()
ax2 = ax.twinx()
peak_hwhm_quotient = mean_amplitude_df.amplitude/mean_hwhm_df.hwhm

peak_hwhm_product = (mean_amplitude_df.amplitude)*(5*mean_hwhm_df.hwhm)


x_axis = np.arange(0,len(epochs))
ax.plot(x_axis,peak_hwhm_quotient,'tab:olive')
ax2.plot(x_axis,peak_hwhm_product,'tab:purple')
# ax.fill_between(x_axis,np.add(peak_hwhm_quotient,plus_sem),np.subtract(peak_hwhm_quotient,minus_sem))
ax.set_ylabel('Aplitude/HWHM quotient (pA/ms)',fontsize=15,labelpad=15)
ax.yaxis.label.set_color('tab:olive')
ax2.set_ylabel('Aplitude*HWHM product (pA*ms)',fontsize=15,labelpad=15)
ax2.yaxis.label.set_color('tab:purple')

ax.set_xticks(np.arange(0,len(epochs))) # REMOVE IN THE FUTURE - PLACED TO AVOID WARNING - IT IS A BUG FROM MATPLOTLIB 3.3.1
ax.set_xticklabels(epochs,fontsize=12)
plt.setp( ax.xaxis.get_majorticklabels(), rotation=45, ha="right" )

fig.suptitle(general_simulation_title,fontsize=25)
plt.tight_layout()
plt.show()




In [ ]:

fig = plt.figure(figsize = (18,7))
ax = plt.axes()

# x_axis = np.arange(0,len(epochs))
ax.scatter(mean_hwhm_df.hwhm/mean_hwhm_df.hwhm[0],mean_amplitude_df.amplitude/mean_amplitude_df.amplitude[0],color='k')
ax.set_xlabel('HWHM (normalized to "Only L4")')
ax.xaxis.label.set_size(15)
ax.set_ylabel('Amplitude (normalized to "Only L4")')
ax.yaxis.label.set_size(15)
plt.axvline(1,ls = '--')
plt.axhline(1,ls = '--')


###########################
#amplified and sharpened
###########################
r1 = patches.Rectangle((1,1), -ax.get_xlim()[1]*2, +ax.get_ylim()[0]*30, color="tab:blue",alpha=0.25)
ax.add_patch(r1)

amplified_sharpened = []
for i,amp in enumerate(mean_amplitude_df.amplitude/mean_amplitude_df.amplitude[0]):
    if amp >1:
        if (mean_hwhm_df.hwhm/mean_hwhm_df.hwhm[0])[i] <1:
            amplified_sharpened.append(epochs[i])
            
# these are matplotlib.patch.Patch properties
amp_sharp_title = dict(boxstyle='round', facecolor="tab:blue",alpha=0.25)
amp_sharp_title_bbox = ax.text(-0.5, 1, 'amplified & sharpened', transform=ax.transAxes, fontsize=15,
        verticalalignment='top', bbox=amp_sharp_title)
amp_sharp_title_bbox_yminmax = get_textbox_yminmax(fig,ax,amp_sharp_title_bbox)


props_epochs = dict(boxstyle='round', facecolor="tab:blue",alpha=0.25)

# place a text box in upper left in axes coords
amp_sharp_bbox = ax.text(-0.5, amp_sharp_title_bbox_yminmax[0]-0.1, '\n'.join(amplified_sharpened) , transform=ax.transAxes, fontsize=12,
                         verticalalignment='top', bbox=props_epochs)
amp_sharp_bbox_yminmax = get_textbox_yminmax(fig,ax,amp_sharp_bbox)

###########################
#dampened and sharpened
###########################
r3 = patches.Rectangle((1,1), -ax.get_xlim()[1]*2, -ax.get_ylim()[1]*2, color="tab:red",alpha=0.25)
ax.add_patch(r3)

dampened_sharpened = []
for i,amp in enumerate(mean_amplitude_df.amplitude/mean_amplitude_df.amplitude[0]):
    if amp <1:
        if (mean_hwhm_df.hwhm/mean_hwhm_df.hwhm[0])[i] <1:
            dampened_sharpened.append(epochs[i])
            
            
# these are matplotlib.patch.Patch properties
damp_sharp_title = dict(boxstyle='round', facecolor="tab:red",alpha=0.25)
damp_sharp_title_bbox = ax.text(-0.5, amp_sharp_bbox_yminmax[0]-0.1, 'dampened & sharpened', transform=ax.transAxes, fontsize=15,
        verticalalignment='top', bbox=damp_sharp_title)
damp_sharp_title_bbox_yminmax = get_textbox_yminmax(fig,ax,damp_sharp_title_bbox)

# print(damp_sharp_title_bbox_yminmax)

props_epochs = dict(boxstyle='round', facecolor="tab:red",alpha=0.25)

# place a text box in upper left in axes coords
damp_sharp_bbox = ax.text(-0.5, damp_sharp_title_bbox_yminmax[0]-0.1, '\n'.join(dampened_sharpened) , transform=ax.transAxes, fontsize=12,
        verticalalignment='top', bbox=props_epochs)
damp_sharp_bbox_yminmax = get_textbox_yminmax(fig,ax,damp_sharp_bbox)
# print(damp_sharp_bbox_yminmax)
###########################
#amplified and un-sharpened
###########################

r2 = patches.Rectangle((1,1), +ax.get_xlim()[1]*2, +ax.get_ylim()[1]*2, color="tab:orange",alpha=0.25)
ax.add_patch(r2)

amplified_unsharpened = []
for i,amp in enumerate(mean_amplitude_df.amplitude/mean_amplitude_df.amplitude[0]):
    if amp >1:
        if (mean_hwhm_df.hwhm/mean_hwhm_df.hwhm[0])[i] >1:
            amplified_unsharpened.append(epochs[i])
            
# these are matplotlib.patch.Patch properties
amp_unsharp_title = dict(boxstyle='round', facecolor="tab:orange",alpha=0.25)
amp_unsharp_title_bbox = ax.text(1.05, 1, 'amplified & unsharpened', transform=ax.transAxes, fontsize=15,
        verticalalignment='top', bbox=amp_unsharp_title)
amp_unsharp_title_bbox = get_textbox_yminmax(fig,ax,amp_unsharp_title_bbox)

props_epochs = dict(boxstyle='round', facecolor="tab:orange",alpha=0.25)

# place a text box in upper left in axes coords
amp_unsharp_bbox = ax.text(1.05, amp_unsharp_title_bbox[0]-0.1, '\n'.join(amplified_unsharpened) , transform=ax.transAxes, fontsize=12,
                           verticalalignment='top', bbox=props_epochs)
amp_unsharp_bbox_yminmax = get_textbox_yminmax(fig,ax,amp_unsharp_bbox)
###########################
#dampened and un-sharpened
###########################
r4 = patches.Rectangle((1,1), +ax.get_xlim()[1]*2, -ax.get_ylim()[1]*2, color="tab:green",alpha=0.25)
ax.add_patch(r4)

dampened_unsharpened = []
for i,amp in enumerate(mean_amplitude_df.amplitude/mean_amplitude_df.amplitude[0]):
    if amp <1:
        if (mean_hwhm_df.hwhm/mean_hwhm_df.hwhm[0])[i] >1:
            dampened_unsharpened.append(epochs[i])
            
            
# these are matplotlib.patch.Patch properties
damp_unsharp_title = dict(boxstyle='round', facecolor="tab:green",alpha=0.25)
damp_unsharp_title_bbox = ax.text(1.05, amp_unsharp_bbox_yminmax[0]-0.1, 'dampened & unsharpened', transform=ax.transAxes, fontsize=15,
        verticalalignment='top', bbox=damp_unsharp_title)
damp_unsharp_title_bbox_yminmax = get_textbox_yminmax(fig,ax,damp_unsharp_title_bbox)



props_epochs = dict(boxstyle='round', facecolor="tab:green",alpha=0.25)

# place a text box in upper left in axes coords
damp_unsharp_bbox = ax.text(1.05, damp_unsharp_title_bbox_yminmax[0]-0.1, '\n'.join(dampened_unsharpened) , transform=ax.transAxes, fontsize=12,
        verticalalignment='top', bbox=props_epochs)
damp_unsharp_bbox_yminmax = get_textbox_yminmax(fig,ax,damp_unsharp_bbox)



fig.suptitle(general_simulation_title,fontsize=25)
plt.tight_layout()
plt.show()


fig.savefig("/".join((parent_folder,simulation_label,'hwhm_amplitude_scatter.pdf')))